In [3]:
from PIL import Image


def resize_image(image_path, output_path, size=(400, 400)):
    try:
        img = Image.open(image_path)
        img = img.resize(size)
        img.save(output_path)
        print("Image resized successfully!")
    except Exception as e:
        print(f"Error resizing image: {e}")


x = 0

while x <= 30:
    input_image_path = "images/images/input_image" + str(x) + ".jpg"
    output_image_path = "output/" + str(x) + ".jpg"
    resize_image(input_image_path, output_image_path)
    x += 1

Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!
Image resized successfully!


In [16]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 50 * 50, 128)  # Adjusted input size to match the output of conv3
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        # Read labels from the text file
        with open(os.path.join(self.root_dir, "labels.txt"), 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                image_name = parts[0]  # First part is the image name
                
                # Join remaining parts to form label (handling cases like "not soap")
                label = " ".join(parts[1:])
                
                # Append file extension to the image path
                image_path = os.path.join(self.root_dir, f"{image_name}.jpg")
                self.image_paths.append(image_path)
                
                # Map string labels to integers
                if label == "soap":
                    self.labels.append(0)
                elif label == "not_soap":
                    self.labels.append(1)
                else:
                    raise ValueError(f"Unknown label: {label}")

                # Print image paths and labels for debugging
                print(f"Image path: {self.image_paths[-1]}, Label: {self.labels[-1]}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

    
# Specify device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify dataset directory
dataset_dir = "output"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
])

# Load dataset
dataset = CustomDataset(root_dir=dataset_dir, transform=transform)

# Define hyperparameters
batch_size = 1
num_epochs = 10
learning_rate = 0.001

# Create data loader
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define number of classes
num_classes = 2 # assuming 2 classes: soap and not soap

# Initialize model
model = SimpleCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(data_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 10 == 9:  # Print every 10 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(data_loader)}], Loss: {running_loss / 10:.3f}")
            running_loss = 0.0
            
        epoch_loss = running_loss / len(data_loader)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {epoch_loss:.3f}")
            
torch.save(model.state_dict(), "path_to_save_model.pth")
print("Training finished!")


Image path: output\0.jpg, Label: 0
Image path: output\1.jpg, Label: 0
Image path: output\2.jpg, Label: 1
Image path: output\3.jpg, Label: 0
Image path: output\4.jpg, Label: 1
Image path: output\5.jpg, Label: 1
Image path: output\6.jpg, Label: 1
Image path: output\7.jpg, Label: 0
Image path: output\8.jpg, Label: 0
Image path: output\9.jpg, Label: 0
Image path: output\10.jpg, Label: 1
Image path: output\11.jpg, Label: 1
Image path: output\12.jpg, Label: 0
Image path: output\13.jpg, Label: 0
Image path: output\14.jpg, Label: 0
Image path: output\15.jpg, Label: 1
Image path: output\16.jpg, Label: 1
Image path: output\17.jpg, Label: 1
Image path: output\18.jpg, Label: 0
Image path: output\19.jpg, Label: 0
Image path: output\20.jpg, Label: 0
Image path: output\21.jpg, Label: 1
Image path: output\22.jpg, Label: 1
Image path: output\23.jpg, Label: 1
Image path: output\24.jpg, Label: 0
Image path: output\25.jpg, Label: 0
Image path: output\26.jpg, Label: 0
Image path: output\27.jpg, Label: 0
Im

Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.076
Epoch [7/10], Average Loss: 0.076
Epoch [7/10], Average Loss: 0.076
Epoch [7/10], Average Loss: 0.079
Epoch [7/10], Average Loss: 0.082
Epoch [7/10], Average Loss: 0.082
Epoch [7/10], Batch [10/31], Loss: 0.255
Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.001
Epoch [7/10], Average Loss: 0.003
Epoch [7/10], Average Loss: 0.008
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Batch [20/31], Loss: 0.040
Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.000
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.013
Epoch [7/10], Average Loss: 0.016
Epoch [7/10], Average Loss: 0.225
Epoch [7/10], Average Loss: 0.225
Epoch [7/10], Average Loss: 0.225


In [5]:
import torch
from torchvision.transforms import ToTensor
from PIL import Image

# Function to resize image
def resize_image(input_image_path, output_image_path):
    image = Image.open(input_image_path)
    resized_image = image.resize((400, 400))
    resized_image.save(output_image_path)
    return output_image_path

# Function to predict class of image
def predict_image(image_path, model):
    # Open the image and apply the same transformations used during training
    image = Image.open(image_path).convert("RGB")
    transform = ToTensor()
    image = transform(image).unsqueeze(0)  # Add batch dimension
    
    # Perform inference
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        
        # Map predicted label index to class name
        class_names = ["soap", "not soap"]
        predicted_class = class_names[predicted.item()]
        
        return predicted_class
x = 0 
while x <= 5:
    input_image_path = "testimages/test_image"+str(x)+".jpg"
    output_image_path = "testimages/resized_image"+str(x)+".jpg"

    # Resize the input image
    resized_image_path = resize_image(input_image_path, output_image_path)

    # Load the trained model
    model = SimpleCNN(num_classes=2)  # Assuming 2 classes: soap and not soap
    model.load_state_dict(torch.load("path_to_save_model.pth"))
    model.eval()

    # Predict the class of the resized image
    predicted_class = predict_image(resized_image_path, model)
    print("Is it soap?:", predicted_class)
    x+=1



Is it soap?: soap
Is it soap?: not soap
Is it soap?: soap
Is it soap?: not soap
Is it soap?: not soap
Is it soap?: soap
